In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("./sdrpy/data/sample_data.csv")

/tmp/ipykernel_24752/1283037797.py:1: DtypeWarning: Columns (17,34,35,38,39,47,54,58,59,86,88,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("./sdrpy/data/sample_data.csv")


In [4]:
df.head()

,_id,Dissemination Identifier,Original Dissemination Identifier,Action type,Event type,Event timestamp,Amendment indicator,Asset Class,Product name,Cleared,...,UPI Underlier Name,Notional amount-Leg 1 mm,Notional amount-Leg 2 mm,CreatedAt,URL,Product type,Product sub type,Tenor,Date,Time
0,65cb4440666de7f4a5284565,901012687,NaN,NEWT,COMP,2024-02-13T06:05:10Z,NaN,IR,OTHERS,I,...,EUR-EURIBOR,78.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,1,2024-02-13,06:05:10
1,65cb4440666de7f4a5284566,901012688,NaN,NEWT,COMP,2024-02-13T06:05:11Z,NaN,IR,OTHERS,I,...,EUR-EURIBOR,69.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,0,2024-02-13,06:05:11
2,65cb4440666de7f4a5284567,901012689,NaN,NEWT,COMP,2024-02-13T06:05:10Z,NaN,IR,OTHERS,I,...,EUR-EURIBOR,47.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,0,2024-02-13,06:05:10
3,65cb4440666de7f4a5284568,901012690,NaN,NEWT,COMP,2024-02-13T06:05:11Z,NaN,IR,OTHERS,I,...,EUR-EURIBOR,400.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,0,2024-02-13,06:05:11
4,65cb4440666de7f4a5284569,901012691,NaN,NEWT,COMP,2024-02-13T06:05:12Z,NaN,IR,OTHERS,I,...,EUR-EURIBOR,320.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,0,2024-02-13,06:05:12


In [47]:
list(df.columns)

['_id',
 'Dissemination Identifier',
 'Original Dissemination Identifier',
 'Action type',
 'Event type',
 'Event timestamp',
 'Amendment indicator',
 'Asset Class',
 'Product name',
 'Cleared',
 'Mandatory clearing indicator',
 'Execution Timestamp',
 'Effective Date',
 'Expiration Date',
 'Maturity date of the underlier',
 'Non-standardized term indicator',
 'Platform identifier',
 'Prime brokerage transaction indicator',
 'Block trade election indicator',
 'Large notional off-facility swap election indicator',
 'Notional amount-Leg 1',
 'Notional amount-Leg 2',
 'Notional currency-Leg 1',
 'Notional currency-Leg 2',
 'Notional quantity-Leg 1',
 'Notional quantity-Leg 2',
 'Total notional quantity-Leg 1',
 'Total notional quantity-Leg 2',
 'Quantity frequency multiplier-Leg 1',
 'Quantity frequency multiplier-Leg 2',
 'Quantity unit of measure-Leg 1',
 'Quantity unit of measure-Leg 2',
 'Quantity frequency-Leg 1',
 'Quantity frequency-Leg 2',
 'Notional amount in effect on associated

In [7]:
# every xccy trade involvinv CAD, with a maturity < 3y from now or > 5y
def get_trades (df, product="xccy", product_type="basis", currency="CAD", maturity=['3y', '5y', "or"], date_range="-1d"):
    if product!=None or product_type!=None:
        df = filter_product(df, product, product_type)
    if currency!=None:
        df = filter_currency(df, currency)
    if maturity!=None:
        df = filter_maturity(df, maturity)
    if date_range!=None:
        df = filter_date_range(df, date_range)
    return df



In [25]:
def filter_product(df, product, product_type):
    product_name_map = pd.read_csv('./sdrpy/data/product_name_map.csv', index_col=0)
    try:
        real_product_name = product_name_map.loc[product].real_name
    except:
        print('ERROR: Could not match product name! Please check data/product_name_map.csv for correct name mapping')
    product_name = str(real_product_name + product_type)
    print(product_name)
    xdf = df[df["Product name"] == product_name]
    return xdf

In [36]:
df2 = filter_product(df, "xccy", "Basis")

InterestRate:CrossCurrency:Basis


In [43]:
def filter_currency(df, currency):
    return df.loc[(df["Notional currency-Leg 1"]==currency) | (df["Notional currency-Leg 2"]==currency)]

In [46]:
df3 = filter_currency(df2, "CAD")

In [ ]:
def filter_by_maturity(df, maturity_dict):
  """
  Filters a dataframe based on the values in the "maturity" column.

  Args:
      df: The pandas dataframe to filter.
      maturity_dict: A dictionary specifying the filtering criteria.
          * Keys can be "lower" or "upper" representing bounds.
          * Values can be numeric (maturity years) or a tuple containing 
            the comparison operator (str, "<", ">", "<=", ">=") and a numeric value.

  Returns:
      A new pandas dataframe containing the filtered rows.
  """
  
  filters = []
  if "lower" in maturity_dict:
    lower_bound, operator = maturity_dict["lower"]
    if isinstance(operator, str):
      filters.append(df["maturity"] >= operator + lower_bound)
    else:
      filters.append(df["maturity"] >= lower_bound)
  
  if "upper" in maturity_dict:
    upper_bound, operator = maturity_dict["upper"]
    if isinstance(operator, str):
      filters.append(df["maturity"] < operator + upper_bound)
    else:
      filters.append(df["maturity"] < upper_bound)
  
  if filters:
    filtered_df = df.query(" & ".join(filters))
  else:
    filtered_df = df.copy()  # return original dataframe if no filters applied
  
  return filtered_df


In [48]:
# Maturity filter
# Maturity less than 3y: [0,3,0]
# Maturity less than 3y, but greater than 2y: [2, 3, "and"]
# Maturity less than 3y, or greater than 5y: [5, 3, "or"]

def get_maturity_column(df):
    df["Effective Date"] = pd.to_datetime(df["Effective Date"])
    df["Expiration Date"] = pd.to_datetime(df["Expiration Date"])
    df["maturity"] = (df["Expiration Date"] - df["Effective Date"]).dt.days / 365

    return df

In [49]:
filter_maturity(df, [])

,_id,Dissemination Identifier,Original Dissemination Identifier,Action type,Event type,Event timestamp,Amendment indicator,Asset Class,Product name,Cleared,...,Notional amount-Leg 1 mm,Notional amount-Leg 2 mm,CreatedAt,URL,Product type,Product sub type,Tenor,Date,Time,Time to Maturity (Years)
0,65cb4440666de7f4a5284565,901012687,NaN,NEWT,COMP,2024-02-13T06:05:10Z,NaN,IR,OTHERS,I,...,78.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,1,2024-02-13,06:05:10,0.246575
1,65cb4440666de7f4a5284566,901012688,NaN,NEWT,COMP,2024-02-13T06:05:11Z,NaN,IR,OTHERS,I,...,69.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,0,2024-02-13,06:05:11,0.249315
2,65cb4440666de7f4a5284567,901012689,NaN,NEWT,COMP,2024-02-13T06:05:10Z,NaN,IR,OTHERS,I,...,47.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,0,2024-02-13,06:05:10,0.249315
3,65cb4440666de7f4a5284568,901012690,NaN,NEWT,COMP,2024-02-13T06:05:11Z,NaN,IR,OTHERS,I,...,400.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,0,2024-02-13,06:05:11,0.509589
4,65cb4440666de7f4a5284569,901012691,NaN,NEWT,COMP,2024-02-13T06:05:12Z,NaN,IR,OTHERS,I,...,320.0,NaN,2024-02-13 10:28:16.286560,https://pddata.dtcc.com/ppd/api/report/intrada...,NaN,NA/Fwd Pr Int Rt Idx EUR,0,2024-02-13,06:05:12,0.246575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20451,65de22ae621ad99ae75d3230,914701909,901565115.0,EROR,NaN,2024-02-13T20:37:35Z,NaN,IR,OTHERS,N,...,8.0,NaN,2024-02-27 17:58:06.746658,https://pddata.dtcc.com/ppd/api/report/intrada...,OTHERS,NA/O P Epn USD,6,2024-02-13,20:37:35,5.756164
20452,65df11cd621ad99ae75d7b5e,915928108,902038075.0,CORR,NaN,2024-02-28T10:52:25Z,NaN,IR,InterestRate:IRSwap:OIS,I,...,5.0,5.0,2024-02-28 10:58:21.186851,https://pddata.dtcc.com/ppd/api/report/intrada...,OIS,OIS,0,2024-02-13,13:40:18,0.783562
20453,65df39cd621ad99ae75d8c1b,916011899,901563757.0,CORR,NaN,2024-02-28T13:45:30Z,NaN,IR,InterestRate:IRSwap:FixedFloat,N,...,50.0,50.0,2024-02-28 13:49:01.856035,https://pddata.dtcc.com/ppd/api/report/intrada...,Vanilla,FxdFlt,5,2024-02-13,19:55:56,5.005479
20454,65df45a4621ad99ae75d95e0,916024382,NaN,NEWT,TRAD,2024-02-13T14:22:57Z,NaN,IR,InterestRate:IRSwap:FixedFloat,N,...,23.0,23.0,2024-02-28 14:39:32.675876,https://pddata.dtcc.com/ppd/api/report/intrada...,Vanilla,FxdFlt,9,2024-02-13,14:22:57,9.602740
